In [4]:
# %%

import pyfolio as pf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
bitcoin_strategy_backtest = pd.read_csv('bitcoin_strategy_backtest.csv', index_col=0, parse_dates=True)
returns = bitcoin_strategy_backtest['returns']
# print(returns.head())
print(pf.__version__)
#plt.show()

bitcoin_strategy_backtest['positions'] = bitcoin_strategy_backtest['positions'].apply(lambda x: x.replace("'", '"'))
bitcoin_strategy_backtest['transactions'] = bitcoin_strategy_backtest['transactions'].apply(lambda x: x.replace("'", '"'))
bitcoin_strategy_backtest['transactions'] = bitcoin_strategy_backtest['transactions'].apply(lambda x: x.replace("datetime.", '"datetime.'))
bitcoin_strategy_backtest['transactions'] = bitcoin_strategy_backtest['transactions'].apply(lambda x: x.replace(")", ')"'))
positions = bitcoin_strategy_backtest[['positions', 'ending_cash']]
position_data = json.loads(positions.iloc[1, 0])
symbol = position_data[0]['symbol']
positions_edited = pd.DataFrame(0, index = positions.index, columns = [[symbol, 'cash']])
for index, row in positions.iterrows():
    position_data = json.loads(row['positions'])
    if position_data:
        positions_edited.loc[index, symbol] = position_data[0]['market_value']   
    positions_edited.loc[index, 'cash'] = float(row['ending_cash'])
#print(positions_edited.head()) 

transactions = bitcoin_strategy_backtest['transactions']
transactions_edited = pd.DataFrame(np.nan, index = transactions.index, columns = [['amount', 'price', 'symbol']])
transactions_edited['symbol'] = transactions_edited['symbol'].astype(str)
# transactions_edited['sid'] = 0
for key, value in transactions.items():
    transactions_data = json.loads(value)
    if transactions_data:
        transactions_edited.loc[key, 'price'] = float(transactions_data[0]['price'])   
        transactions_edited.loc[key, 'amount'] = int(transactions_data[0]['amount'])
        transactions_edited.loc[key, 'symbol'] = symbol
        # transactions_edited.loc[key, 'txn_dollars'] = - float(transactions_data[0]['price']) * int(transactions_data[0]['amount'])
print('transactions_edited: ', transactions_edited.tail())
transactions_edited_2 = transactions_edited.dropna()
print('transactions_edited_2: ', transactions_edited_2.tail())

transactions_edited.iloc[-1]['amount'] = - transactions_edited_2.iloc[-1]['amount']
transactions_edited.iloc[-1]['price'] = json.loads(positions.iloc[-1, 0])[0]['last_price']
transactions_edited.iloc[-1]['symbol'] = symbol
print('transactions_edited: ', transactions_edited.tail())

transactions_edited_3 = transactions_edited.dropna()
# 如果列名是多级的，降为单级
if isinstance(transactions_edited_3.columns, pd.MultiIndex):
    transactions_edited_3.columns = [col[0] for col in transactions_edited_3.columns]
if not isinstance(transactions_edited_3.index, pd.DatetimeIndex):
    transactions_edited_3.index = pd.to_datetime(transactions_edited_3.index)
# 只保留 pyfolio 需要的列
transactions_edited_3 = transactions_edited_3[['symbol', 'amount', 'price']]

# 检查数据类型
transactions_edited_3['symbol'] = transactions_edited_3['symbol'].astype(str)
transactions_edited_3['amount'] = transactions_edited_3['amount'].astype(float)
transactions_edited_3['price'] = transactions_edited_3['price'].astype(float)

print('returns: ', returns.head())
print('returns: ', returns.tail())
print('positions_edited: ', positions_edited.head())
print('positions_edited: ', positions_edited.tail())
print('positions_edited: ', positions_edited.dtypes)
print('transactions_edited_3: ', transactions_edited_3.head())
print('transactions_edited_3: ', transactions_edited_3.tail())
print('transactions_edited_3: ', transactions_edited_3.dtypes)
# pf.create_returns_tear_sheet(returns,
#                          positions=positions_edited,
#                          transactions=transactions_edited)
#                          slippage=1)
# help(pf.create_full_tear_sheet)
# pf.create_full_tear_sheet(returns,
#                        positions=positions_edited,
#                        transactions=transactions_edited,
#                        round_trips=True,
#                        bootstrap=False,
#                        estimate_intraday=False,
#                        slippage=10)
# plt.show()

0.9.3+199.g55a87fb
transactions_edited:             amount price symbol
date                          
2024-12-13    NaN   NaN    nan
2024-12-14    NaN   NaN    nan
2024-12-15    NaN   NaN    nan
2024-12-16    NaN   NaN    nan
2024-12-17    NaN   NaN    nan
transactions_edited_2:             amount    price  symbol
date                              
2024-02-29 -778.0  61101.5  BTCUSD
2024-04-14  735.0  65074.0  BTCUSD
2024-07-21 -693.0  68101.5  BTCUSD
2024-09-07  867.0  54112.5  BTCUSD
2024-10-17 -385.0  67472.5  BTCUSD
transactions_edited:             amount price symbol
date                          
2024-12-13    NaN   NaN    nan
2024-12-14    NaN   NaN    nan
2024-12-15    NaN   NaN    nan
2024-12-16    NaN   NaN    nan
2024-12-17    NaN   NaN    nan
returns:  date
2017-11-29    0.000000
2017-11-30   -0.001955
2017-12-01    0.013201
2017-12-02    0.091065
2017-12-03   -0.001972
Name: returns, dtype: float64
returns:  date
2024-12-13    0.006011
2024-12-14   -0.001195
2024-12-15   